In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'isaiah-jackson-3', 'ayo-dosunmu', 'david-duke', 'sandro-mamukelashvili', 'corey-kispert', 'aaron-wiggins',
    'matthew-mcclung', 'kai-jones',  'santi-aldama', 'cade-cunningham', 'jalen-suggs','evan-mobley', 'franz-wagner', 
    'davion-mitchell', 'scottie-barnes','moses-moody', 'ziaire-williams', 'tre-mann', 'josh-christopher', 
    'nahshon-hyland', 'jalen-johnson-24', 'jaden-springer', 'cameron-thomas', 'miles-mcbride', 'charles-bassey', 
    'chris-duarte', 'james-bouknight', 'jeremiah-robinson-earl', 'sharife-cooper', 'brandon-bostonjr', 
    'isaiah-livers','dayron-sharpe', 'keon-johnson', 'marcus-zegarowski', 'matthew-hurt', 'trey-murphyiii', 
    'ochai-agbaji', 'joshua-primo', 'kessler-edwards', 'neemias-queta', 'jason-preston',  'jt-thor', 
    'quentin-grimes', 'joe-wieskamp', 'javonte-smart', 'luka-garza', 'trendon-watford', 'austin-reaves', 
    'greg-brown-9', 'dalano-banton', 'david-johnson-13', 'jericho-sims', 'raiquan-gray', 'jared-butler']

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2021')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-2].isdigit():
            player = player[:-3]
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        if player[-3:] == 'iii':
            player = player.replace('iii', ' III')
        playerlist.append(player)
        
    except:
        continue
    try:
        player = player.replace('-', '+')
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2021'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/35))*2.33), 2)
        year_list.append(year)
        
    except:
        age_list.append(0)
        year_list.append(.85)

age_list[0] = 19.5
year_list[0] = 1.03
age_list[1] = 21.4
year_list[1] = .86
age_list[2] = 21.7
year_list[2] = .81
age_list[3] = 22.1
year_list[3] = .78
age_list[4] = 22.3
year_list[4] = .76
age_list[5] = 22.5
year_list[5] = .74
age_list[6] = 22.5
year_list[6] = .74
age_list[7] = 20.4
year_list[7] = .94
age_list[8] = 20.5
year_list[8] = .93
age_list[-1] = 20.9
year_list[-1] = .89

table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.25) + (table['AST']*2) +
(table['BLK']*2) + (table['STL']*3) + (table['3P']*2) + (table['3PA']) + (table['SOS']/2)) * table['Year']


table["Player Grade"] = table["Player Grade"]*1.75
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']

table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Cade Cunningham,2020-21,19.8,Oklahoma State,9.78,27,35.4,.438,2.3,5.7,0.400,0.846,6.2,3.5,1.6,0.8,20.1,,99.8
2,Sharife Cooper,2020-21,20.1,Auburn,9.28,12,33.1,.391,1.1,4.8,0.228,0.825,4.3,8.1,1.0,0.3,20.2,,98.8
3,Cameron Thomas,2020-21,19.7,LSU,9.14,29,34.0,.406,2.3,7.2,0.325,0.882,3.4,1.4,0.9,0.2,23.0,,88.4
4,Nahshon Hyland,2020-21,20.8,VCU,4.76,24,31.9,.447,2.9,7.8,0.371,0.862,4.7,2.1,1.9,0.2,19.5,,85.9
5,Santi Aldama,2020-21,20.5,Loyola (MD),-3.47,17,35.0,.513,1.9,5.1,0.368,0.686,10.1,2.3,1.0,1.7,21.2,,84.6
6,Evan Mobley,2020-21,20.0,USC,10.65,33,33.9,.578,0.4,1.2,0.300,0.694,8.7,2.4,0.8,2.9,16.4,,83.3
7,Quentin Grimes,2020-21,21.2,Houston,5.37,30,32.8,.406,3.3,8.3,0.403,0.788,5.7,2.0,1.4,0.3,17.8,,82.6
8,Jared Butler,2020-21,20.9,Baylor,7.40,30,30.3,.471,2.6,6.2,0.416,0.780,3.3,4.8,2.0,0.4,16.7,,81.5
9,Moses Moody,2020-21,19.1,Arkansas,7.18,32,33.8,.427,1.8,5.1,0.358,0.812,5.8,1.6,1.0,0.7,16.8,,81.5
10,Tre Mann,2020-21,20.4,Florida,9.82,24,32.4,.459,1.9,4.7,0.402,0.831,5.6,3.5,1.4,0.1,16.0,,81.2
